<a href="https://colab.research.google.com/github/FCosmin27/ocs/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random

In [2]:
upper_bound = 100
lower_bound = -100
no_of_variables = 10

In [3]:
def check_x(x):
    for i in range(len(x)):
        if x[i] > upper_bound:
            x[i] = upper_bound
        elif x[i] < lower_bound:
            x[i] = lower_bound
    return x

In [4]:
x = np.random.normal(size=no_of_variables)

x = check_x(x)
print(x)

[ 1.39647607  0.05952313  0.0163228  -0.20732874 -0.05416054  1.77906818
  2.21661906  0.0672337  -0.44894654  0.68630529]


In [5]:
def f1(x):
    return np.sum(np.square(x))

In [6]:
def f2(x):
    result = 0
    for i in range(no_of_variables):
        tmp = 0
        for j in range(i):
            tmp += x[j]

        result += tmp**2

    return result

In [7]:
def f3(x):
    result = 0
    for i in range(1, no_of_variables):
        result += 1e6 ** ((i-1)/(no_of_variables-1)) * x[i]**2

    return result

In [8]:
def f4(x):
    result = 0
    for i in range(no_of_variables):
        tmp = 0
        for j in range(i):
            tmp += x[j]

        result += tmp**2

    return result * (1 + 0.4 * np.abs(np.random.randn()))

In [9]:
def f5(x):
    A = np.random.randint(-500, 501, size=(no_of_variables, no_of_variables))

    while np.linalg.det(A) == 0:
        A = np.random.randint(-500, 501, size=(no_of_variables, no_of_variables))

    return np.max(np.abs(x*A - A))

In [ ]:
print("f1(x):", f1(x))
print("f2(x):", f2(x))
print("f3(x):", f3(x))
print("f4(x):", f4(x))
print("f5(x):", f5(x))

In [10]:
def calculate_min_x(func, no_of_agents, x, es, alpha):
    min_x = x
    for _ in range(es):
        xm = min_x + alpha * np.random.normal(size=no_of_variables)
        xm = check_x(xm)
        if func(xm) < func(min_x):
            min_x = xm
    return min_x

In [11]:
def get_population(number_of_variables, population_size):
    return np.random.normal(size=(population_size, number_of_variables))

In [12]:
initial_alpha = 0.5
no_of_variables = 10
population_size = 5
iterations = 100
functions = [f1, f2, f3, f4, f5]
population = get_population(no_of_variables, population_size)

In [ ]:
def population_v2_adaptive(population, function):
    for iter_no in range(iterations):
        alpha = initial_alpha / (iter_no + 1)
        new_population = []

        for agent in population:
            new_agents = [agent + alpha * np.random.normal(size=no_of_variables) for _ in range(5)]
            best_agent = min(new_agents, key=lambda x: function(x))
            new_population.append(best_agent)

        if sum([function(agent) for agent in new_population]) < sum([function(agent) for agent in population]):
            population = new_population

    return population

In [ ]:
print("f1(x):", population_v2_adaptive(population, f1))
print("f2(x):", population_v2_adaptive(population, f2))
print("f3(x):", population_v2_adaptive(population, f3))
print("f4(x):", population_v2_adaptive(population, f4))
print("f5(x):", population_v2_adaptive(population, f5))

In [ ]:
for idx, population in enumerate(population_v2_adaptive()):
    print("Function", idx+1)
    for agent in population:
        print(agent, f1(agent))


In [13]:
def pc_function(iter_no):
    return max(0.5, 1 / (1 + iter_no))

In [14]:
def pm_function(iter_no):
    return max(0.01, 0.1 / (1 + iter_no))

In [15]:
def sphere_function(x):
    return np.sum(np.square(x))

In [17]:
def CGA_adaptiveV1_greedy(pc_function, pm_function, population, function, no_of_variables):
    NFE = 0
    max_NFE = 1000
    iter_no = 0

    while NFE < max_NFE:
        iter_no += 1
        pc = pc_function(iter_no)
        pm = pm_function(iter_no)

        new_population = []
        for i in range(len(population)):
            parent1 = population[i]
            parent2 = population[np.random.randint(len(population))]

            if np.random.rand() < pc:
                crossover_point = np.random.randint(1, no_of_variables - 1)
                child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            else:
                child = parent1.copy()

            if np.random.rand() < pm:
                mutation_idx = np.random.randint(no_of_variables)
                child[mutation_idx] += np.random.normal()

            new_population.append(child)

        new_population.sort(key=function)
        best_parent = min(population, key=function)
        new_population[-1] = best_parent

        population = new_population

        NFE += len(new_population)

    return population

In [18]:
final_population = CGA_adaptiveV1_greedy(pc_function, pm_function, population, sphere_function, no_of_variables)
print("Best solution (CGA):", min(final_population, key=sphere_function))


Best solution (CGA): [-0.49011166  0.07608145 -0.69306099 -1.15093236 -0.16511177  0.05238892
 -0.32661151  0.01738802  0.38317513  0.32700305]


In [19]:
def pc_function_fitness(fitness):
    return 0.5 + 0.4 * fitness

In [20]:
def pm_function_fitness(fitness):
    return 0.1 - 0.08 * fitness

In [22]:
def CGA_adaptiveV2_greedy(pc_function, pm_function, population, function, no_of_variables):
    NFE = 0
    max_NFE = 1000

    while NFE < max_NFE:
        fitness_values = np.array([function(individual) for individual in population])
        max_fitness = np.max(fitness_values)
        min_fitness = np.min(fitness_values)

        normalized_fitness = (max_fitness - fitness_values) / (max_fitness - min_fitness + 1e-10)

        pc = pc_function(normalized_fitness)
        pm = pm_function(normalized_fitness)

        new_population = []
        for i in range(len(population)):
            parent1 = population[i]
            parent2 = population[np.random.randint(len(population))]

            if np.random.rand() < pc[i]:
                crossover_point = np.random.randint(1, no_of_variables - 1)
                child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            else:
                child = parent1.copy()

            if np.random.rand() < pm[i]:
                mutation_idx = np.random.randint(no_of_variables)
                child[mutation_idx] += np.random.normal()

            new_population.append(child)

        new_population.sort(key=function)
        best_parent = min(population, key=function)
        new_population[-1] = best_parent

        population = new_population

        NFE += len(new_population)

    return population


In [23]:
final_population_v2 = CGA_adaptiveV2_greedy(pc_function_fitness, pm_function_fitness, population, f1, no_of_variables)
print("Best solution (CGA V2):", min(final_population_v2, key=f1))


Best solution (CGA V2): [ 0.01931239 -0.30544388  0.11654601 -0.04336698 -0.12141563  0.05238892
 -0.32661151  0.01738802  0.05382677  0.32700305]


In [25]:
def RGA_4(population, function, pc=0.9, pm=0.1, max_NFE=1000):
    NFE = 0
    no_of_variables = population.shape[1]
    while NFE < max_NFE:
        new_population = []
        for _ in range(len(population) // 2):
            parent1, parent2 = population[np.random.choice(len(population), 2, replace=False)]

            if np.random.rand() < pc:
                alpha = 0.1
                child1 = np.random.uniform(np.minimum(parent1, parent2) - alpha * np.abs(parent1 - parent2),
                                           np.maximum(parent1, parent2) + alpha * np.abs(parent1 - parent2))
                child2 = np.random.uniform(np.minimum(parent1, parent2) - alpha * np.abs(parent1 - parent2),
                                           np.maximum(parent1, parent2) + alpha * np.abs(parent1 - parent2))
            else:
                child1, child2 = parent1.copy(), parent2.copy()

            if np.random.rand() < pm:
                mutation_idx = np.random.randint(no_of_variables)
                child1[mutation_idx] += np.random.normal()
            if np.random.rand() < pm:
                mutation_idx = np.random.randint(no_of_variables)
                child2[mutation_idx] += np.random.normal()

            child1 = np.clip(child1, -10, 10)
            child2 = np.clip(child2, -10, 10)

            new_population.extend([child1, child2])

        combined_population = np.vstack((population, new_population))
        combined_population = sorted(combined_population, key=function)
        population = np.array(combined_population[:len(population)])

        NFE += len(new_population)

    return population

In [27]:
final_population_v2 = RGA_4(population, f1)
print("Best solution (RGA_4):", min(final_population_v2, key=f1))

Best solution (CGA V2): [-3.05743887e-02  6.22209232e-02 -2.99545163e-02  1.16265863e-02
  6.75334746e-03 -8.92766834e-03  2.83640722e-02 -3.70288975e-04
 -1.44694861e-04  3.59688935e-01]


In [29]:
def DE_best_1_exp(population, function, F=0.8, CR=0.9, max_NFE=1000):
    NFE = 0
    population_size, no_of_variables = population.shape

    while NFE < max_NFE:
        new_population = []

        for i in range(population_size):
            best = min(population, key=function)
            r1, r2 = population[np.random.choice(population_size, 2, replace=False)]
            mutant = best + F * (r1 - r2)

            trial = np.copy(population[i])
            start_idx = np.random.randint(no_of_variables)
            for j in range(no_of_variables):
                if np.random.rand() < CR or j == start_idx:
                    trial[j] = mutant[j]

            trial = np.clip(trial, -10, 10)

            if function(trial) < function(population[i]):
                new_population.append(trial)
            else:
                new_population.append(population[i])

        population = np.array(new_population)
        NFE += population_size

    return population


In [30]:
final_population_v2 = DE_best_1_exp(population, f1)
print("Best solution (DE_best_1_exp):", min(final_population_v2, key=f1))

Best solution (DE_best_1_exp): [-0.23885957  0.221679    0.14638907 -0.26745006  0.19292714  0.03546124
 -0.30634016 -0.19258842 -0.00337336  0.19196741]
